Step 1 : Importing the required packages

In [34]:
import pandas as pd
import numpy as np
# Bokeh Packages
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, curdoc, export_png,output_file, save
from bokeh.models import HoverTool,ColumnDataSource, Text, BoxAnnotation, ColorBar, Toggle
from bokeh.layouts import gridplot, layout, column, row

In [35]:
from bokeh.models import LinearColorMapper
from bokeh.palettes import OrRd 
from bokeh.sampledata.us_counties import data as counties

 Step 2 : Importing the CSV files

In [36]:
# Importing the HCAHPS Data
hcahps=pd.read_csv("HCAHPS-Hospital (1).csv")

C:\Users\hamid\AppData\Local\Temp\ipykernel_4612\3910223051.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  hcahps=pd.read_csv("HCAHPS-Hospital (1).csv")


In [37]:
# import hospital info
hsp=pd.read_csv("HospitalData_Revised.csv")

Step 3.0: Data wrangling
Step 3.1: Create a common list of facility names between two datasets

In [38]:
# Selecting HCAHPS of California
hcahps_ca=hcahps[hcahps["State"]=="CA"]

In [39]:
# filtering just the linear mean value
df=hcahps_ca[hcahps_ca['HCAHPS Linear Mean Value'].astype(str).str.isdigit()]

In [40]:
# clean | define the type of the values
df_clean=df.astype({"Facility Name":"category","HCAHPS Question":"category","HCAHPS Linear Mean Value":int})
df_clean.sort_values("Facility Name",inplace=True)

In [41]:
# drop the duplicates from the list of facililty names
Facility= df_clean["Facility Name"]
Facility.drop_duplicates(inplace=True)

In [42]:
# Build a list to compare with the other dataset
Facility.tolist()
# Facility.sort_values(inplace=True)
F=Facility.str.title()

In [43]:
#  Select the rows which has common values with the list in previous block
G=hsp[hsp["Facility Name"].isin(F)]
# Build the final common list between two datasets
Facility_common=G["Facility Name"]
Facility_common.drop_duplicates(inplace=True)
G1=G.drop_duplicates("Facility Name")
G1.sort_values("Facility Name",ignore_index=True,inplace=True)

C:\Users\hamid\AppData\Local\Temp\ipykernel_4612\3365128406.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Facility_common.drop_duplicates(inplace=True)
C:\Users\hamid\AppData\Local\Temp\ipykernel_4612\3365128406.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  G1.sort_values("Facility Name",ignore_index=True,inplace=True)


Step 3.2: Create a dataframe for the overview of the two datasets

In [44]:
# pull out the overview rating of hospital from HCAHPS
Facility_common_C=Facility_common.str.upper()
df_common = df_clean[df_clean["Facility Name"].isin(Facility_common_C)]
I=df_common[df_common["HCAHPS Question"]=="Overall hospital rating - linear mean score"]
I.sort_values("Facility Name",ignore_index=True,inplace=True)
# I.reindex(list(range(0,I.shape[0]-1)))
H=pd.DataFrame({'Facility Name':G1["Facility Name"],
                'County': I["County Name"],
                'Stories': G1['Stories'],
                'Building Code Year': G1['Building Code Year'],
                'Year Completed': G1['Year Completed'],
                'Latitude': G1["Latitude"],
                'Longitude': G1['Longitude'],
                'HCAHPS Score':I["HCAHPS Linear Mean Value"]})
H

C:\Users\hamid\AppData\Local\Temp\ipykernel_4612\680760185.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  I.sort_values("Facility Name",ignore_index=True,inplace=True)


,Facility Name,County,Stories,Building Code Year,Year Completed,Latitude,Longitude,HCAHPS Score
0,Adventist Health And Rideout,YUBA,6.0,2007,2018,39.138222,-121.594360,83
1,Adventist Health Bakersfield,KERN,5.0,1998,2010,35.383377,-119.019699,86
2,Adventist Health Clearlake,LAKE,NaN,2001,2015,38.936192,-122.620243,86
3,Adventist Health Hanford,KINGS,1.0,2007,2016,36.325715,-119.668399,87
4,Adventist Health Howard Memorial,MENDOCINO,2.0,2007,2015,39.390464,-123.339290,93
...,...,...,...,...,...,...,...,...
132,Valley Presbyterian Hospital,LOS ANGELES,6.0,1995,2005,34.194946,-118.463211,87
133,Ventura County Medical Center,VENTURA,6.0,2010,2019,34.277264,-119.252989,87
134,Washington Hospital,ALAMEDA,4.0,2001,2021,37.558010,-121.980602,89
135,Watsonville Community Hospital,SANTA CRUZ,NaN,1992,2004,36.928230,-121.793502,80


In [45]:
# indexing based on the facility name
Ind=H["Facility Name"]
H.set_index(Ind,inplace=True)
# output_notebook()
H

,Facility Name,County,Stories,Building Code Year,Year Completed,Latitude,Longitude,HCAHPS Score
Facility Name,,,,,,,,
Adventist Health And Rideout,Adventist Health And Rideout,YUBA,6.0,2007,2018,39.138222,-121.594360,83
Adventist Health Bakersfield,Adventist Health Bakersfield,KERN,5.0,1998,2010,35.383377,-119.019699,86
Adventist Health Clearlake,Adventist Health Clearlake,LAKE,NaN,2001,2015,38.936192,-122.620243,86
Adventist Health Hanford,Adventist Health Hanford,KINGS,1.0,2007,2016,36.325715,-119.668399,87
Adventist Health Howard Memorial,Adventist Health Howard Memorial,MENDOCINO,2.0,2007,2015,39.390464,-123.339290,93
...,...,...,...,...,...,...,...,...
Valley Presbyterian Hospital,Valley Presbyterian Hospital,LOS ANGELES,6.0,1995,2005,34.194946,-118.463211,87
Ventura County Medical Center,Ventura County Medical Center,VENTURA,6.0,2010,2019,34.277264,-119.252989,87
Washington Hospital,Washington Hospital,ALAMEDA,4.0,2001,2021,37.558010,-121.980602,89


Step 4.0: Create the basic figures

This scatter plot will help up to identify the outliers to define the case studies we want to focus on

In [46]:
x=H["Facility Name"]
y=H["HCAHPS Score"]

output_notebook()
# prepare data source
source = ColumnDataSource(data=dict(
    x=H["Year Completed"],
    y=H["HCAHPS Score"],
    f=H["Facility Name"],
))
TOOLTIPS = [
    ("Facility", "@f"),
    ("Year Completed", "@x"),
    ("Score","@y"),
    ("Index","$index")
]
TOOLS="hover,save"
p = figure(title= "Scatter plot", x_axis_label='x', y_axis_label='y',
           tools=TOOLS,
           tooltips=TOOLTIPS)
p.circle('x', 'y', size=15,fill_alpha=0.6, source=source)

low_box = BoxAnnotation(top=H["HCAHPS Score"].mean(), fill_alpha=0.3, fill_color='#0072B2')
p.add_layout(low_box)

p.xaxis.axis_label="Year"
p.yaxis.axis_label="HCAHPS Score"

# Make it transparent for web and png export
p.background_fill_alpha = 0
p.border_fill_alpha=0



# Toggle
toggle1 = Toggle(label="Below Average", button_type="success", active=False)
toggle1.js_link('active', low_box, 'visible')

layout = column(toggle1, p)

# output_file("plot.html")
# save(p)
show(layout)


Loading BokehJS ...

Step 4.0: Let's dive into some case studies of facilities which have been built recently but have poor score

 First: Colvis Community Medical Center | 2022 | score: 82 | Index: 21

In [47]:
# Step 4.1 Prepare Data
c=H["Facility Name"][21].upper()
case1 =df_clean[df_clean["Facility Name"]== c]

In [48]:
cc1=pd.DataFrame({c:case1["HCAHPS Linear Mean Value"]})
scoreType=["Recommend Hospital", "Cleanliness","Care Transition","Discharge Information","Nurse Communication", "Communication about medicines",
          "Quietness","Overall hospital rating","Doctor communication","Staff responsiveness"]
# cc1.reindex(scoreType)
# c1=pd.DataFrame({"Recommend Hospital":case1["HCAHPS Linear Mean Value"][1]})
# c1
c1=cc1.transpose(copy=True)
c1.columns=scoreType
c1

,Recommend Hospital,Cleanliness,Care Transition,Discharge Information,Nurse Communication,Communication about medicines,Quietness,Overall hospital rating,Doctor communication,Staff responsiveness
CLOVIS COMMUNITY MEDICAL CENTER,83,86,77,83,85,65,75,82,86,75


In [49]:
# Step 4.2 Bar chart
# first figure
x=list(c1.columns)
y=list(case1["HCAHPS Linear Mean Value"])
data_cp1=ColumnDataSource(dict(x=x,
              y=y,))
TOOLS = "pan,hover,save"
case1_fig1=figure(title=c,tools=TOOLS,x_range=list(c1.columns),tooltips=[("HCAHPS Value","@x"),("Score","@y")],
                 sizing_mode="stretch_width", max_width=1000, height=600)
case1_fig1.vbar(x="x",top="y",width=0.5,source=data_cp1)
case1_fig1.xaxis.major_label_orientation = 1
# text = Text(x=10, y=10, text="Some text",text_font_size="100pt",text_color="red")
# case1_fig1.add_glyph(text)

# Set the axis label
# case1_fig1.xaxis.axis_label = "HCAHPS Value"
case1_fig1.yaxis.axis_label = "Score"

# Make it transparent for web and png export
case1_fig1.background_fill_alpha = 0
case1_fig1.border_fill_alpha=0

# Second figure
# case1_fig2 = figure()
# case1_fig2.square([1, 2, 3], [4, 5, 6])
# add two figure into a grid
# grid=gridplot([[case1_fig1,case1_fig2]])
# show(case1_fig1)
output_file("case1_fig1.html")
save(case1_fig1)

'C:\\Users\\hamid\\EMAT_InteractiveData\\Final Project\\case1_fig1.html'

Step 5.0 County Heatmap

In [50]:
# Step 5.1 Prepare data
# Create County Scores
# CountyScore=pd.DataFrame({"County":H.groupby(["County"]).mean()["HCAHPS Score"]
CountyScore= H.groupby(["County"]).mean()["HCAHPS Score"]
# CountyScore.sort_index(ascending=True)
CountyScore.index = CountyScore.index.map(lambda x: x.title())

C:\Users\hamid\AppData\Local\Temp\ipykernel_4612\332738199.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  CountyScore= H.groupby(["County"]).mean()["HCAHPS Score"]


In [51]:
# Step 5.2 data for the dict and palette and color mapping
palette = list(reversed(OrRd[9]))
counties = {
    code: county for code, county in counties.items() if county["state"] == "ca"
}

county_xs = [county["lons"] for county in counties.values()]
county_ys = [county["lats"] for county in counties.values()]

county_names = [county['name'] for county in counties.values()]
county_rates = CountyScore
data=dict(
    x=county_xs,
    y=county_ys,
    name=county_names,
    rate=county_rates,
)
color_mapper = LinearColorMapper(palette=palette, low=min(data['rate']), high=max(data['rate']))
for county_name in county_names:
    if county_name not in county_rates:
        county_rates[county_name] = 0

In [52]:
# Step 5.3 The actual map with interactivity features
TOOLS = "pan,wheel_zoom,reset,save"

heatmap_fig = figure(
    title="HCAHPS Score in CA, 2021", tools=TOOLS,
    x_axis_location=None, y_axis_location=None,
    tooltips=[
        ("Name", "@name"), ("HCAHPS Score", "@rate%"), ("(Long, Lat)", "($x, $y)")],sizing_mode='scale_width')
heatmap_fig.grid.grid_line_color = None
heatmap_fig.hover.point_policy = "follow_mouse"

heatmap_fig.patches('x', 'y', source=data,
          fill_color={'field': 'rate', 'transform': color_mapper},
          fill_alpha=0.7, line_color="black", line_width=.2)

# Make it transparent for web and png export
heatmap_fig.background_fill_alpha = 0
heatmap_fig.border_fill_alpha=0
heatmap_fig.outline_line_alpha = 0

# add color bar
color_bar = ColorBar(color_mapper=color_mapper, location=(0, 0),border_line_color=None,label_standoff=12)
color_bar.background_fill_alpha = 0
heatmap_fig.add_layout(color_bar, 'right')

# show(heatmap_fig)
output_file("heatmap_fig.html")
save(heatmap_fig)

'C:\\Users\\hamid\\EMAT_InteractiveData\\Final Project\\heatmap_fig.html'